In [133]:
import glob, os
import xml.etree.ElementTree as etree
import pandas as pd
import csv

cwd = os.getcwd()
print(cwd)

C:\Users\Nous\Desktop\Marion\Etalab\DECP\data


In [134]:
def remplacer(node):
    try : 
        return node.text
    except : 
        return ''

In [135]:
pathData = r"C:\Users\Nous\Desktop\Marion\Etalab\DECP\data"
pathSave = r"C:\Users\Nous\Desktop\Marion\Etalab\DECP"

In [136]:
files = []

os.chdir(pathData)
for file in glob.glob("*.xml"):
    #print(file)
    files.append(file)

In [137]:
def traiterContrat (contrat, typeContrat, nomFichierSource, numContrat) :
    
    # Cas où l'id a été oublié
    try : 
        idContrat = contrat.find('id')
        idContrat = idContrat.text
    except : # Création d'un id unique --> a retravailler
        idContrat = nomFichierSource + '_' + str(numContrat)
    
    if typeContrat == "marche" :
        uidContrat = idContrat + '_' + contrat.find('acheteur').find('id').text
        
    if typeContrat == "contrat-concession" :
        uidContrat = idContrat + '_' +  contrat.find('autoriteConcedante').find('id').text

    #### Infos générales relatives au marché
    infosG = {'idContrat' : idContrat, 'uidContrat' : uidContrat, 'typeContrat' : typeContrat}
    
    infosG['nature'] = remplacer(contrat.find('nature'))
    infosG['objet'] = remplacer(contrat.find('objet'))
    infosG['codeCPV'] = remplacer(contrat.find('codeCPV'))
    infosG['procedure'] = remplacer(contrat.find('procedure'))
    infosG['lieuExecTypeCode'] = remplacer(contrat.find('lieuExecution').find('typeCode'))
    infosG['lieuExecCode'] = remplacer(contrat.find('lieuExecution').find('code'))
    infosG['lieuExecNom'] = remplacer(contrat.find('lieuExecution').find('nom'))
    infosG['dureeMois'] = remplacer(contrat.find('dureeMois'))
    infosG['dateNotification'] = remplacer(contrat.find('dateNotification'))[:10]
    infosG['datePublicationDonnees'] = remplacer(contrat.find('datePublicationDonnees'))[:10]
    infosG['montant'] = remplacer(contrat.find('montant'))
    infosG['formePrix'] = remplacer(contrat.find('formePrix'))
    
    # Partie spécifique aux concessions
    infosG['dateSignature'] = remplacer(contrat.find('dateSignature'))[:10]
    infosG['dateDebutExecution'] = remplacer(contrat.find('dateDebutExecution'))[:10]
    infosG['valeurGlobale'] = remplacer(contrat.find('valeurGlobale'))
    infosG['montantSubventionPublique'] = remplacer(contrat.find('montantSubventionPublique'))
    
    #### Infos relatives à l'acheteur / autorité concédante
    infosA = {'idContrat' : idContrat, 'uidContrat' : uidContrat}
    
    if typeContrat == "marche" :
        infosA['idAcheteur'] = remplacer(contrat.find('acheteur').find('id'))
        infosA['nomAcheteur'] = remplacer(contrat.find('acheteur').find('nom'))
        
    if typeContrat == "concession" :
        infosA['idAcheteur'] = remplacer(contrat.find('autoriteConcedante').find('id'))
        infosA['nomAcheteur'] = remplacer(contrat.find('autoriteConcedante').find('nom'))
        
    
    #### Infos relatives aux titulaires / Concessionnaires / En attendant de le faire en une fois
    infosT = []
    for titulaire in contrat.findall('titulaires/titulaire') :
        infosTitulaire = {'idContrat' : idContrat, 'uidContrat' : uidContrat}
        infosTitulaire['typeIdentifiant'] = remplacer(titulaire.find('typeIdentifiant'))
        infosTitulaire['idTitulaire'] = remplacer(titulaire.find('id'))
        infosTitulaire['denominationSocialeTitulaire'] = remplacer(titulaire.find('denominationSociale'))
        infosT.append(infosTitulaire)
    for titulaire in contrat.findall('concessionnaires/concessionnaire') : 
        infosTitulaire = {'idContrat' : idContrat, 'uidContrat' : uidContrat}
        infosTitulaire['typeIdentifiant'] = remplacer(titulaire.find('typeIdentifiant'))
        infosTitulaire['idTitulaire'] = remplacer(titulaire.find('id'))
        infosTitulaire['denominationSocialeTitulaire'] = remplacer(titulaire.find('denominationSociale'))
        infosT.append(infosTitulaire)
                    
    return({'infosG' : infosG, 'infosA' : infosA, 'infosT' : infosT})


In [138]:
infosG_new = []
infosA_new = []
infosT_new = []

In [139]:
for file in files :
    #print("Traitement de : " + file)
    try : 
        tree = etree.parse(path + '/' + file)
        root = tree.getroot()
    except : 
        print("Problème avec le fichier : " + file)
        continue
        
    id_file = file
    c = 0
    for contrat in root : 
        c = c+1
        informations = traiterContrat(contrat, contrat.tag, id_file, c)
        #print(informations)
        infosG_new.append(informations['infosG'])
        infosA_new.append(informations['infosA'])
        for element in informations['infosT'] : # Boucle car plusieurs titulaires possibles
            infosT_new.append(element)

Problème avec le fichier : DECP-21590663700016-2019-03-14-01.xml
Problème avec le fichier : DECP-21670180500014-2019-03-14-01.xml
Problème avec le fichier : DECP-21670180500287-2019-03-14-01.xml
Problème avec le fichier : DECP-21690123100011-2019-03-14-01.xml
Problème avec le fichier : DECP-21890025600015-2019-03-14-01.xml
Problème avec le fichier : DECP-21920026800015-2019-03-14-01.xml
Problème avec le fichier : DECP-24850056300031-2019-03-14-01.xml
Problème avec le fichier : DECP-25590116700031-2019-03-14-01.xml
Problème avec le fichier : DECP-26490044000019-2019-03-14-01.xml
Problème avec le fichier : DECP-28590001500055-2019-03-14-01.xml


In [140]:
# Passage en tableau et réagencement des colonnes
infosG_new = pd.DataFrame(infosG_new)
infosG_new = infosG_new[['idContrat', 'uidContrat', 'typeContrat', 'nature', 'objet', 'codeCPV', 'procedure', 
                         'lieuExecTypeCode', 'lieuExecCode', 'lieuExecNom',
                         'dureeMois', 'dateNotification', 'datePublicationDonnees', 'montant', 'formePrix', 
                         'dateSignature', 'dateDebutExecution', 'valeurGlobale', 'montantSubventionPublique']]

infosA_new = pd.DataFrame(infosA_new)
infosA_new = infosA_new[['idContrat', 'uidContrat', 'idAcheteur', 'nomAcheteur']]

infosT_new = pd.DataFrame(infosT_new)
infosT_new = infosT_new[['idContrat', 'uidContrat', 'typeIdentifiant', 'idTitulaire', 'denominationSocialeTitulaire']]

In [141]:
infosG_new.head(2)

,idContrat,uidContrat,typeContrat,nature,objet,codeCPV,procedure,lieuExecTypeCode,lieuExecCode,lieuExecNom,dureeMois,dateNotification,datePublicationDonnees,montant,formePrix,dateSignature,dateDebutExecution,valeurGlobale,montantSubventionPublique
0,201981MA0300,201981MA0300_13000056500015,marche,Accord-cadre,18MA03,31682000-0,Appel d'offres ouvert,Code commune,MDPH,Orvault - St Nazaire,12,2018-11-30,2019-04-05,200000.0000,Ferme,,,,
1,2019I1703000,2019I1703000_13001898900017,marche,Marché,MAINTENANCE DES INSTALLATIONS DE CHAUFFAGE / V...,50700000-2,Appel d'offres ouvert,Code commune,INO,Angers Le Mans Nantes Tours,38,2018-12-17,2019-03-19,153900.0000,Ferme,,,,


In [142]:
infosA_new.head(2)

,idContrat,uidContrat,idAcheteur,nomAcheteur
0,201981MA0300,201981MA0300_13000056500015,13000056500015,MDPH
1,2019I1703000,2019I1703000_13001898900017,13001898900017,INOVALYS


In [143]:
infosT_new.head(2)

,idContrat,uidContrat,typeIdentifiant,idTitulaire,denominationSocialeTitulaire
0,201981MA0300,201981MA0300_13000056500015,SIRET,54210765113030,ENGIE
1,2019I1703000,2019I1703000_13001898900017,SIRET,85480074501125,AXIMA CONCEPT


In [144]:
infosG_new.to_csv(pathSave + '\Contrats.csv', index=False, sep= '|', quoting=csv.QUOTE_NONNUMERIC)
infosA_new.to_csv(pathSave + '\Acheteurs.csv', index=False, sep= '|', quoting=csv.QUOTE_NONNUMERIC)
infosT_new.to_csv(pathSave + '\Titulaires.csv', index=False, sep= '|', quoting=csv.QUOTE_NONNUMERIC)